# Search using query rules

<a target="_blank" href="https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/05-query-rules.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

이 대화형 노트북은 공식 [Elasticsearch Python 클라이언트](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html)를 사용하여 쿼리 규칙을 사용하는 방법을 소개합니다. [query rules API](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-rules-apis.html)를 사용하여 Elasticsearch에 쿼리 규칙을 저장하고 [rule_query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-rule-query.html)를 사용하여 쿼리합니다.

## Elasticsearch/Kibana 실행

먼저 Elasticsearch/Kibana를 실행하세요. 단, Elasticsearch 버전이 **8.10.0** 이상인지 확인하세요. 다른 모든 설정은 기본값으로 둡니다.

## 패키지 설치 및 모듈 가져오기

시작하려면 Python 클라이언트를 사용하여 Elastic 배포에 연결해야 합니다.

먼저 `elasticsearch` Python 클라이언트를 설치해야 합니다.

In [1]:
!pip install -qU elasticsearch

## Elasticsearch 클라이언트 초기화

이제 [Elasticsearch Python 클라이언트](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/index.html) 인스턴스를 생성합니다.   
Eleasticsearch URL, 사용자 ID, 사용자 Password, 인증서 경로 정보가 필요하며, 인증서 경로는 (auto_install 시) `kibana.yml` 하단에 기재되어 있습니다.
Elasticsearch에 로컬로 연결하는 방법에 대해  [자세히 알아보세요](https://www.elastic.co/guide/en/elasticsearch/client/python-api/current/connecting.html#_verifying_https_with_certificate_fingerprints_python_3_10_or_later)

In [2]:
from elasticsearch import Elasticsearch
from getpass import getpass

ES_URL = input('Elasticsearch URL: ')
ES_ID = "elastic" 
ES_PW = getpass('elastic user PW: ')
CERT_PATH = input('Elasticsearch cer 파일 경로: ')

# Create the client instance
client = Elasticsearch(
    ES_URL,
    basic_auth=(ES_ID, ES_PW),
    ca_certs=CERT_PATH
)

클라이언트가 이 테스트에 연결되었는지 확인합니다.

In [3]:
print(client.info())

{'name': 'DESKTOP-2AKM6J7', 'cluster_name': 'elasticsearch', 'cluster_uuid': '5tEVK4EEQ0WPUotyTCvXYg', 'version': {'number': '8.11.1', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '6f9ff581fbcde658e6f69d6ce03050f060d1fd0c', 'build_date': '2023-11-11T10:05:59.421038163Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


## 일부 테스트 데이터 색인화

클라이언트가 설정되어 Elastic 배포에 연결되었습니다.
이제 Elasticsearch 쿼리의 기본 사항을 테스트하려면 일부 데이터가 필요합니다.
다음 필드와 함께 작은 제품 색인을 사용합니다.:

- `name`
- `description`
- `price`
- `currency`
- `plug_type`
- `voltage`

### 인덱스 테스트 데이터

다음 명령어를 실행하여 일부 샘플 데이터를 업로드하세요.

In [4]:
import json
import os

cwd = os.getcwd()
url = cwd + "/query-rules-data.json"
response = open(url)
docs = json.loads(response.read())

operations = []
for doc in docs:
    operations.append({"index": {"_index": "products_index", "_id": doc["id"]}})
    operations.append(doc["content"])
client.bulk(index="products_index", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 2096, 'items': [{'index': {'_index': 'products_index', '_id': 'us1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products_index', '_id': 'uk1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products_index', '_id': 'eu1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'products_index', '_id': 'preview1', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1, 'status': 201}}]})

## 테스트 데이터 검색

먼저, 신뢰할 수 있는 무선 충전기를 찾기 위해 데이터를 검색해 보겠습니다.

데이터를 검색하기 전에 Elasticsearch의 원시 JSON 응답을 사람이 더 읽기 쉬운 형식으로 출력하는 몇 가지 편의 함수를 정의하겠습니다.

In [5]:
def pretty_response(response):
    if len(response['hits']['hits']) == 0:
        print('Your search returned no results.')
    else:
        for hit in response['hits']['hits']:
            id = hit['_id']
            score = hit['_score']
            name = hit['_source']['name']
            description = hit['_source']['description']
            price = hit["_source"]["price"]
            currency = hit["_source"]["currency"]
            plug_type = hit["_source"]["plug_type"]
            voltage = hit["_source"]["voltage"]
            pretty_output = (f"\nID: {id}\nName: {name}\nDescription: {description}\nPrice: {price}\nCurrency: {currency}\nPlug type: {plug_type}\nVoltage: {voltage}\nScore: {score}")
            print(pretty_output)

def pretty_ruleset(response):
    print("Ruleset ID: " + response['ruleset_id'])
    for rule in response['rules']:
        rule_id = rule['rule_id']
        type = rule['type']
        print(f"\nRule ID: {rule_id}\n\tType: {type}\n\tCriteria:")
        criteria = rule['criteria']
        for rule_criteria in criteria:
            criteria_type = rule_criteria['type']
            metadata = rule_criteria['metadata']
            values = rule_criteria['values']
            print(f"\t\t{metadata} {criteria_type} {values}")
        ids = rule['actions']['ids']
        print(f"\tPinned ids: {ids}")

다음으로 검색을 수행합니다:

In [6]:
response = client.search(index="products_index", query={
    "multi_match": {
        "query": "reliable wireless charger for iPhone",
        "fields": [ "name^5", "description" ]
    }
})

pretty_response(response)


ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled charging speeds with elegant design, it promises both rapid and dependable energy for your devices. Embrace the future of wireless charging.
Price: 18.0
Currency: EUR
Plug type: C
Voltage: 230V
Score: 14.5004

ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to charge on-the-go, up to two full charges. Pre-order yours today!
Price: 36.0
Currency: USD
Plug type: ['B', 'C', 'G']
Voltage: ['230V', '120V']
Score: 1.0668641

ID: us1
Name: PureJuice Pro
Description: PureJuice Pro: Experience the pinnacle of wireless charging. Blending rapid charging tech with sleek design, it e

응답에서 볼 수 있듯이 유럽의 결과가 1위를 차지했습니다. 예를 들어, 내 검색자가 플러그와 사양이 다른 미국이나 영국에서 온다는 것을 알고 있는 경우 이는 바람직하지 않을 수 있습니다.

여기서 쿼리 규칙이 도움이 될 수 있습니다!

## 규칙 만들기

별도로 사용자가 어느 국가에서 왔는지 알고 있다고 가정해 보겠습니다(아마도 IP 주소 또는 로그인한 사용자 계정 정보의 지리적 위치). 이제 우리는 사람들이 `wireless charger(무선 충전기)`라는 문구가 포함된 항목을 검색할 때 해당 정보를 기반으로 무선 충전기를 강화하는 쿼리 규칙을 만들고 싶습니다.

In [9]:
client.query_ruleset.put(ruleset_id="promotion-rules", rules=[
    {
      "rule_id": "us-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["us"]
        }
      ],
      "actions": {
        "ids": [
          "us1"
        ]
      }
    },
    {
      "rule_id": "uk-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["uk"]
        }
      ],
      "actions": {
        "ids": [
          "uk1"
        ]
      }
    }
  ])

ObjectApiResponse({'result': 'updated'})

이러한 규칙이 일치하려면 다음 중 하나가 충족되어야 합니다.:

- `my_query` 에는 "wireless charger"라는 문자열이 포함되어 있습니다. " *AND* `country`는 "us"입니다.
- `my_query` 에는 "wireless charger"라는 문자열이 포함되어 있습니다. " *AND* `country`는 "uk"입니다.

API를 사용하여 규칙 세트를 볼 수도 있습니다(가독성을 위해 또 다른 `pretty_ruleset` 함수 사용):

In [10]:
response = client.query_ruleset.get(ruleset_id="promotion-rules")
pretty_ruleset(response)

Ruleset ID: promotion-rules

Rule ID: us-charger
	Type: pinned
	Criteria:
		my_query contains ['wireless charger']
		country exact ['us']
	Pinned ids: ['us1']

Rule ID: uk-charger
	Type: pinned
	Criteria:
		my_query contains ['wireless charger']
		country exact ['uk']
	Pinned ids: ['uk1']


다음으로, rule_query를 사용하여 위와 동일한 organic query를 사용하지만 쿼리 규칙을 추가하여 검색을 수행합니다:

In [11]:
response = client.search(index="products_index", query={
      "rule_query": {
          "organic": {
              "multi_match": {
                  "query": "reliable wireless charger for iPhone",
                  "fields": [ "name^5", "description" ]
              }
          },
          "match_criteria": {
            "my_query": "reliable wireless charger for iPhone",
            "country": "us"
          },
          "ruleset_id": "promotion-rules"
      }
})

pretty_response(response)


ID: us1
Name: PureJuice Pro
Description: PureJuice Pro: Experience the pinnacle of wireless charging. Blending rapid charging tech with sleek design, it ensures your devices are powered swiftly and safely. The future of charging is here.
Price: 15.0
Currency: USD
Plug type: B
Voltage: 120v
Score: 1.7014122e+38

ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled charging speeds with elegant design, it promises both rapid and dependable energy for your devices. Embrace the future of wireless charging.
Price: 18.0
Currency: EUR
Plug type: C
Voltage: 230V
Score: 14.5004

ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to cha

규칙 쿼리는 먼저 표시하려는 문서를 향상시킵니다.

규칙을 적용하려면 규칙의 모든 기준이 일치해야 합니다. 예를 들어 `country`를 'ca'로 업데이트하면 두 쿼리 규칙 모두 적용되지 않으며 수정되지 않은 organic 결과 set이 반환됩니다.

In [12]:
response = client.search(index="products_index", query={
      "rule_query": {
          "organic": {
              "multi_match": {
                  "query": "reliable wireless charger for iPhone",
                  "fields": [ "name^5", "description" ]
              }
          },
          "match_criteria": {
            "my_query": "reliable wireless charger for iPhone",
            "country": "ca"
          },
          "ruleset_id": "promotion-rules"
      }
})

pretty_response(response)


ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled charging speeds with elegant design, it promises both rapid and dependable energy for your devices. Embrace the future of wireless charging.
Price: 18.0
Currency: EUR
Plug type: C
Voltage: 230V
Score: 14.5004

ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to charge on-the-go, up to two full charges. Pre-order yours today!
Price: 36.0
Currency: USD
Plug type: ['B', 'C', 'G']
Voltage: ['230V', '120V']
Score: 1.0668641

ID: us1
Name: PureJuice Pro
Description: PureJuice Pro: Experience the pinnacle of wireless charging. Blending rapid charging tech with sleek design, it e

단일 규칙 쿼리에 여러 규칙을 적용하는 것도 가능합니다. 최신 모델의 사전 주문 페이지에 프로모션 결과가 항상 고정되도록 규칙 세트를 업데이트해 보겠습니다.

규칙은 순서대로 적용되므로 규칙 세트 시작 부분에 선주문 문서를 넣습니다.

In [11]:
client.query_ruleset.put(ruleset_id="promotion-rules", rules=[
    {
      "rule_id": "preorder",
      "type": "pinned",
      "criteria": [
        {
          "type": "always"
        }
      ],
      "actions": {
        "ids": [
          "preview1"
        ]
      }
    },
    {
      "rule_id": "us-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["us"]
        }
      ],
      "actions": {
        "ids": [
          "us1"
        ]
      }
    },
    {
      "rule_id": "uk-charger",
      "type": "pinned",
      "criteria": [
        {
          "type": "contains",
          "metadata": "my_query",
          "values": ["wireless charger"]
        },
        {
          "type": "exact",
          "metadata": "country",
          "values": ["uk"]
        }
      ],
      "actions": {
        "ids": [
          "uk1"
        ]
      }
    }
  ])

ObjectApiResponse({'result': 'updated'})

이제 영국에서 검색하면 선주문 페이지가 먼저 고정되고, 영국 결과가 두 번째로 고정된 후 organic search 결과가 반환됩니다

In [12]:
response = client.search(index="products_index", query={
      "rule_query": {
          "organic": {
              "multi_match": {
                  "query": "reliable wireless charger for iPhone",
                  "fields": [ "name^5", "description" ]
              }
          },
          "match_criteria": {
            "my_query": "reliable wireless charger for iPhone",
            "country": "uk"
          },
          "ruleset_id": "promotion-rules"
      }
})

pretty_response(response)


ID: preview1
Name: PureJuice Pro - Pre-order next version
Description: Newest version of the PureJuice Pro wireless charger, coming soon! The newest model of the PureJuice Pro boasts a 2x faster charge than the current model, and a sturdier cable with an eighteen month full warranty. We also have a battery backup to charge on-the-go, up to two full charges. Pre-order yours today!
Price: 36.0
Currency: USD
Plug type: ['B', 'C', 'G']
Voltage: ['230V', '120V']
Score: 1.7014124e+38

ID: uk1
Name: PureJuice Pro - UK Compatible
Description: PureJuice Pro: Redefining wireless charging. Seamlessly merging swift charging capabilities with a refined aesthetic, it guarantees your devices receive rapid and secure power. Welcome to the next generation of charging.
Price: 20.0
Currency: GBP
Plug type: G
Voltage: 230V
Score: 1.7014122e+38

ID: eu1
Name: PureJuice Pro - Wireless Charger suitable for European plugs
Description: PureJuice Pro: Elevating wireless charging. Combining unparalleled chargin

These examples start to show the power of promoting documents based on contextual query metadata. For more information on how to get started using query rules, check out our [blog post](https://www.elastic.co/blog/introducing-query-rules-elasticsearch-8-10) and [search guide](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-using-query-rules.html).